In [ ]:
import sys
import os
import numpy as np


In [ ]:
from DiffusionAnalysis.loaders import DatDirectoryStructureLoader
from DiffusionAnalysis.analysis import TracerMSDAnalyser
from DiffusionAnalysis.trajectory import DisplacementTrajectory




In [ ]:
# your_code.py
from DiffusionAnalysis.examples.config import FILE_PATH_SECRET

loader = DatDirectoryStructureLoader(FILE_PATH_SECRET, structures_slice=slice(2500, 5000, 1), md_timestep=0.1, md_time_unit='ps')

In [ ]:
traj = DisplacementTrajectory(loader)

In [ ]:
traj.generate_displacement_trajectory(show_progress=True)
# # #save as a pickle (not implemented in the class)
import pickle
# with open('displacement_traj.pickle', 'wb') as f:
#       pickle.dump(traj, f)

#load the pickle
with open('displacement_traj.pickle', 'rb') as f:
    traj = pickle.load(f)
    


In [ ]:
msd_analysis = TracerMSDAnalyser(traj)

# Assuming you have an instance of MSDAnalysis called 'msd_analysis'

# Calculate the 3D MSD
msd_3d = msd_analysis.calculate_msd(tracer_specs=['H'], return_3d_msd=True)

msd_3d_framework = msd_analysis.calculate_msd(tracer_specs=['H'], framework_specs=['He'], correct_drift=True, return_3d_msd=True)
fig1 = msd_analysis.plot_msd((msd_3d), title='3D MSD Plot (Drift Corrected)', legend_loc='upper left', skip_points=4)
fig1_log = msd_analysis.plot_msd((msd_3d), title='3D MSD Plot (Log Scale) (Drift Corrected)', legend_loc='upper left', skip_points=4, log_scale=True)


#plot 3d vs 3d framework drift corrected msd
difference = msd_3d - msd_3d_framework
fig1_diff = msd_analysis.plot_msd((difference), title='3D MSD vs 3D MSD (Drift Corrected) Error', legend_loc='upper left', skip_points=4)


# Calculate the MSD for each direction (x, y, z)
msd_x, msd_y, msd_z = msd_analysis.calculate_msd(tracer_specs=['H'],return_3d_msd=False)

# Plot the MSD for each direction
fig2 = msd_analysis.plot_msd((msd_x, msd_y, msd_z), labels=['x','y' ,'z'], title='X,Y,Z MSD Plot', legend_loc='upper left', skip_points=4)
fig2.show()
# Calculate and plot the MSD along a non-normalized lattice vector for host atoms, correcting for framework drift
lattice_vector = np.array([2, 0, 0])  # Example non-normalized lattice vector
msd_along_lattice = msd_analysis.calculate_msd(lattice_vector = lattice_vector)
fig3 = msd_analysis.plot_msd(msd_along_lattice, labels=f'MSD Along Lattice Vector {lattice_vector}', title='MSD Along Lattice Vector [2,0,0] (Drift Uncorrected)', skip_points=4)
fig3.show()


In [ ]:
from DiffusionAnalysis.analysis import VanHoveAnalyser
import numpy as np
import matplotlib.pyplot as plt


# Create a VanHoveAnalysis object
van_hove_analyzer = VanHoveAnalyser(traj)

# Set the parameters for the analysis
tau_values = [1, 5, 10, 500]  # Time lags in number of timesteps
r_range = (0, 5)  # Range of distances to consider
n_bins = 100  # Number of bins for the histogram

self_data = []
self_data = van_hove_analyzer.calculate_van_hove(tau_values, r_range, n_bins, mode='self',type_a_specs=['H'])

self_fig = van_hove_analyzer.plot_van_hove(self_data, mode='self', title='Self-part of the Van Hove Correlation Function')

import tracemalloc


distinct_data = []
distinct_data = van_hove_analyzer.calculate_van_hove(tau_values, r_range, n_bins, mode='distinct', type_a_specs=['H'], type_b_specs=['He'])

# Plot the distinct-part of the van Hove correlation function
distinct_fig = van_hove_analyzer.plot_van_hove(distinct_data, mode='distinct', title='Distinct-part of the Van Hove Correlation Function', include_normalization=True)


In [ ]:
from DiffusionAnalysis.analysis import TMSDAnalyser

# Create an instance of the tMSDAnalysis class
tmsd_analysis = TMSDAnalyser(traj)

# Define the minimum and maximum time lag values (in steps) and the number of points - start at 0
tau_values = np.unique(np.logspace(0, 3.35, num=500, dtype=int))

# Calculate tMSD
time_lag_values, tMSD_values = tmsd_analysis.calculate_tMSD(tau_values, tracer_specs=['H'], framework_specs=['He','Li'], overlapping=True, correct_drift=True, use_3d=False, lattice_vector=[1,0,0])

# Plot the tMSD data
temperature = 500  # Example temperature value
label = f'{temperature} K'
fig = tmsd_analysis.plot_tMSD(time_lag_values, tMSD_values, label)

fig2 = tmsd_analysis.plot_tMSD_exponent(time_lag_values, tMSD_values, label, window_size=1) 




In [ ]:
time_lags = [1, 100, 23]  # Example time lags
num_bins = 40

fig = msd_analysis.plot_displacement_distribution(tracer_specs=['H'],
                                                  time_lags=time_lags,
                                                  num_bins=num_bins, use_3d=False, lattice_vector=[1, 0, 0])
fig.gca().set_xlim(-3, 3)

fig = msd_analysis.plot_displacement_distribution(tracer_specs=['H'],
                                                  time_lags=time_lags,
                                                  num_bins=num_bins,
                                                  use_3d=True,)

fig.gca().set_xlim(0, 6)

plt.show()

In [ ]:
from DiffusionAnalysis.analysis import COMMSDAnalyser

com_analysis = COMMSDAnalyser(traj)

# Calculate the 3D COM MSD
com_msd_3d_no_drift = com_analysis.calculate_com_msd(tracer_specs=['H'], framework_specs=['He','Li'], correct_drift=False, return_3d_msd=True)
com_msd_3d = com_analysis.calculate_com_msd(tracer_specs=['H'], framework_specs=['He','Li'], correct_drift=True, return_3d_msd=True)


fig1_log = com_analysis.plot_com_msd(com_msd_3d, title='3D COM MSD Plot (Log Scale) (Drift Corrected)', legend_loc='upper left', skip_points=1, log_scale=True)
# plot the difference
difference = com_msd_3d - com_msd_3d_no_drift
fig1 = com_analysis.plot_com_msd(com_msd_3d, title='3D COM MSD Plot (Drift Corrected)', legend_loc='upper left', skip_points=1)

fig1_diff = com_analysis.plot_com_msd(com_msd_3d_no_drift, title='3D COM MSD (Drift Uncorrected)', legend_loc='upper left', skip_points=1)
# # Calculate the COM MSD for each direction (x, y, z)
# com_msd_x, com_msd_y, com_msd_z = com_analysis.calculate_com_msd(species_specs=['H'], return_3d_msd=False)

diff = com_msd_3d - com_msd_3d_no_drift
fig2 = com_analysis.plot_com_displacement_distribution(tracer_specs=['H'],framework_specs=['He','Li'], correct_drift=True, time_lags=[1, 2, 3], num_bins=20, use_3d=True)
fig2 = com_analysis.plot_com_displacement_distribution(tracer_specs=['H'],framework_specs=['He','Li'], correct_drift=True, time_lags=[1, 2, 3], num_bins=20, use_3d=False, lattice_vector=[1, 0, 0])




In [ ]:
from DiffusionAnalysis.analysis import COMTMSDAnalyser

com_msd_analysis = COMTMSDAnalyser(traj)

# Assuming you have an instance of COMTMSDAnalyser called 'com_msd_analysis'

# Calculate the 3D tMSD
#logspace 1 to 1000 in 50 steps adn no repeats .. have to be ints
#tau_values = np.unique(np.logspace(0, 3.1, num=100, dtype=int))
tMSD_3d = com_msd_analysis.calculate_tMSD(tau_values=tau_values, tracer_specs=['H'],correct_drift = False, use_3d=True, overlapping=True)

fig_uncorrected = com_msd_analysis.plot_tMSD(tMSD_3d[0], tMSD_3d[1], title='3D tMSD Plot (Drift Uncorrected)', legend_loc='upper left',label='500K' )
tMSD_3d_framework = com_msd_analysis.calculate_tMSD(tau_values=tau_values, tracer_specs=['H'], framework_specs= ['He','Li'], correct_drift=True, use_3d=True, overlapping=True)

fig1 = com_msd_analysis.plot_tMSD(tMSD_3d_framework[0], tMSD_3d_framework[1], title='3D tMSD Plot (Drift Corrected)', legend_loc='upper left',label='500K' )

fig1_log = com_msd_analysis.plot_tMSD(tMSD_3d_framework[0], tMSD_3d_framework[1], title='3D tMSD Plot (Log Scale) (Drift Corrected)', legend_loc='upper left', log_scale=True, label='500K')

fig_exponent = com_msd_analysis.plot_tMSD_exponent(tMSD_3d_framework[0], tMSD_3d_framework[1], label='500K', average_window_size=50)
# Plot 3d vs 3d framework drift corrected tMSD
difference = tMSD_3d[1] - tMSD_3d_framework[1]

fig1_diff = com_msd_analysis.plot_tMSD(tMSD_3d[0], difference, title='3D tMSD vs 3D tMSD (Drift Corrected) Error', legend_loc='upper left', label='500K')

# # Calculate the tMSD for each direction (x, y, z)
# tMSD_x = com_msd_analysis.calculate_tMSD(tau_values=tau_values, tracer_specs=['H'], use_3d=False)
# tMSD_y = com_msd_analysis.calculate_tMSD(tau_values=tau_values, tracer_specs=['H'], use_3d=False, lattice_vector=np.array([0, 1, 0]))
# tMSD_z = com_msd_analysis.calculate_tMSD(tau_values=tau_values, tracer_specs=['H'], use_3d=False, lattice_vector=np.array([0, 0, 1]))

# # Plot the tMSD for each direction
# fig2 = com_msd_analysis.plot_tMSD(tMSD_x[0], tMSD_x[1], label='x', title='X,Y,Z tMSD Plot', legend_loc='upper left')
# com_msd_analysis.plot_tMSD(tMSD_y[0], tMSD_y[1], label='y', fig=fig2)
# com_msd_analysis.plot_tMSD(tMSD_z[0], tMSD_z[1], label='z', fig=fig2)

# fig2.show()

# # Calculate and plot the tMSD along a non-normalized lattice vector for host atoms, correcting for framework drift
# lattice_vector = np.array([2, 0, 0])  # Example non-normalized lattice vector

# tMSD_along_lattice = com_msd_analysis.calculate_tMSD(tau_values=tau_values, lattice_vector=lattice_vector)

# fig3 = com_msd_analysis.plot_tMSD(tMSD_along_lattice[0], tMSD_along_lattice[1], label=f'tMSD Along Lattice Vector {lattice_vector}', title='tMSD Along Lattice Vector [2,0,0] (Drift Uncorrected)')

# fig3.show()

In [ ]:
num_H_atoms = len(traj.atom_indices_map.get_indices('H'))
volume = traj.unique_lattice_vectors.volume * 1e-24  # Volume in cm^3
print(volume)
print(num_H_atoms)

In [ ]:
from DiffusionAnalysis.analysis import DiffusionCoefficientAnalyser
# convcert 0.1ps to seconds
time_step = 10**-12
com_diffusion_coefficient = DiffusionCoefficientAnalyser.calculate_com_diffusion_coefficient(tMSD_3d_framework[1], tMSD_3d_framework[0],step_size = time_step ,dimension=3, num_atoms=num_H_atoms)
com_conductivity = DiffusionCoefficientAnalyser.calculate_conductivity(com_diffusion_coefficient, temperature=500, num_atoms=num_H_atoms, volume=volume)

In [ ]:
# plot diffusion coefficient with log x and y scale
plt.plot(tMSD_3d_framework[0]*10**-12, com_diffusion_coefficient)
plt.xscale('log')
plt.yscale('log')

plt.show()
plt.plot(tMSD_3d_framework[0]*10**-12,com_conductivity)
plt.xscale('log')
plt.yscale('log')
print(com_conductivity[-1])

In [ ]:

print(com_conductivity[-1])
room_temp_conductivity = DiffusionCoefficientAnalyser.convert_conductivity_to_different_temperature(com_conductivity, activation_energy = 5.3e-20, temperature = 500, new_temperature=273)

plt.plot(tMSD_3d_framework[0]*10**-12,room_temp_conductivity)
plt.xscale('log')
plt.yscale('log')

print(room_temp_conductivity[-1])
#print last 10 tau values
print(tau_values[-10:])

In [ ]:
tau_values = np.unique(np.logspace(0, 3.3, num=100, dtype=int))
print(tau_values[-10:])